In [18]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse

def ODtoA(OD,INDEX):
    INDEX=INDEX
    M=pd.concat([ OD[['O']],OD[['D']]],axis=0)
    #M.fillna(0)
    M=M.fillna(0)
    M['OD']=M['O']+M['D']

    m = n = 2085
    Mat = [[0] * m] * n
    Mat_np=np.array(Mat)
    for i in range(len(OD)):
        O=OD.iloc[i,:]['O_id']
        D=OD.iloc[i,:]['D_id']
        Mat_np[O,D]=OD.iloc[i,:]['flow']
      #  if(Mat_np[O,D]>5000):
         #   print(Mat_np[O,D])
    # Diag is zero
    row, col = np.diag_indices_from(Mat_np)
    Mat_np[row,col] = 0

    m = n = 2085
    Mat_sym = [[0] * m] * n
    Mat_sym=np.array(Mat_sym)

    for i in range(2085):
        for j in range(2085):
            Mat_sym[i,j]=Mat_np[i,j]+Mat_np[j,i]


    # convert to sparse matrix representation 
    A = csr_matrix(Mat_sym)
    print("Sparse matrix: \n",A)

    m = 1
    n=2085
   
    sparse.save_npz('D:/GNN/OCD/A_'+INDEX+'.npz', A)  #保存
   

In [36]:
## X_DeepWalk

import networkx as nx # 图数据挖掘

import random # 随机数
from tqdm import tqdm # 进度条
from scipy.sparse import csr_matrix
from scipy import sparse

# 数据可视化
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签  
plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号


def get_randomwalk(node, path_length):
    '''
    输入起始节点和路径长度，生成随机游走节点序列
    '''
    
    random_walk = [node]
    
    for i in range(path_length-1):
        # 汇总邻接节点
        temp = list(G.neighbors(node))
        temp = list(set(temp) - set(random_walk))    
        if len(temp) == 0:
            break
        # 从邻接节点中随机选择下一个节点
        random_node = random.choice(temp)
        random_walk.append(random_node)
        node = random_node
        
    return random_walk
def ODtoX_deepWalk(OD,INDEX):
    G = nx.from_pandas_edgelist(OD, "O", "D", edge_attr=True, create_using=nx.Graph())
    INDEX=INDEX
    all_nodes = list(G.nodes())

    gamma = 10 # 每个节点作为起始点生成随机游走序列个数
    walk_length = 5 # 随机游走序列最大长度

    random_walks = []

    for n in tqdm(all_nodes): # 遍历每个节点
        for i in range(gamma): # 每个节点作为起始点生成gamma个随机游走序列
            random_walks.append(get_randomwalk(n, walk_length))
    from gensim.models import Word2Vec # 自然语言处理
    model = Word2Vec(vector_size=256, # Embedding维数
                     window=4, # 窗口宽度
                     sg=1, # Skip-Gram
                     hs=0, # 不加分层softmax
                     negative=10, # 负采样
                     alpha=0.03,  # 初始学习率
                     min_alpha=0.0007, # 最小学习率
                     seed=14 # 随机数种子
                    )

    # 用随机游走序列构建词汇表
    model.build_vocab(random_walks, progress_per=2)

    # 训练（耗时1分钟左右）
    model.train(random_walks, total_examples=model.corpus_count, epochs=50, report_delay=1)

    X = model.wv.vectors
    X_pd=pd.DataFrame(X)

    nodes_pd=pd.DataFrame(all_nodes)
    nodes_pd.columns=['node']
    X_deep=pd.concat([nodes_pd,X_pd],axis=1)

    dic_id=pd.read_csv('dic_id.csv')
    dic_id.columns=['cbg','id']
    dic_id

    dic_id.set_index('cbg',inplace=True) ## 只有重新定义一列索引，后续与社区发现结果进行concat的时候，才能对得上
    X_deep.set_index('node',inplace=True) ## 只有重新定义一列索引，后续与社区发现结果进行concat的时候，才能对得上
    result=pd.concat([dic_id,X_deep],axis=1)

    result=result.fillna(0)
    result[result.isnull().T.any()]


    Xdeep_np=np.array(result)
    Xdeep = csr_matrix(Xdeep_np)

   # print("Sparse matrix: \n",Xdeep)
    print('Finish')
    sparse.save_npz('D:/GNN/OCD/X_deep_'+INDEX+'.npz', Xdeep)  #保存


In [33]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import preprocessing
import numpy as np
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from gensim.models import Word2Vec
import warnings
import collections
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import networkx as nx # 图数据挖掘
# 数据分析
import pandas as pd
import numpy as np
import random # 随机数
from tqdm import tqdm # 进度条

# 数据可视化
import matplotlib.pyplot as plt
%matplotlib inline

def ODtoX_node2vec(OD,INDEX):
    OD=OD.rename(columns={'flow':'weight'})

    plt.rcParams['font.sans-serif']=['SimHei']  # 用来正常显示中文标签  
    plt.rcParams['axes.unicode_minus']=False  # 用来正常显示负号

    G = nx.from_pandas_edgelist(OD, "O", "D",edge_attr=['weight'], create_using=nx.Graph())
    nx.get_edge_attributes(G, "weight")
    G_ste = StellarGraph.from_networkx(G)
    walk_length = 100  # maximum length of a random walk to use throughout this notebook
    _, weights = G_ste.edges(include_edge_weight=True)

    wt, cnt = np.unique(weights, return_counts=True)
    rw = BiasedRandomWalk(G_ste)
    weighted_walks = rw.run(
        nodes=G_ste.nodes(),  # root nodes
        length=walk_length,  # maximum length of a random walk
        n=10,  # number of random walks per root node
        p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
        q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
        weighted=True,  # for weighted random walks
        seed=42,  # random seed fixed for reproducibility
    )
    print("Number of random walks: {}".format(len(weighted_walks)))
    weighted_model = Word2Vec(
        weighted_walks, vector_size=256, window=5, min_count=0, sg=1, workers=1
    )
    X = weighted_model.wv.vectors
    X.shape
    all_nodes = list(G.nodes())

    X_pd=pd.DataFrame(X)

    nodes_pd=pd.DataFrame(all_nodes)
    nodes_pd.columns=['node']
    X_deep=pd.concat([nodes_pd,X_pd],axis=1)

    dic_id=pd.read_csv('dic_id.csv')
    dic_id.columns=['cbg','id']
    dic_id

    dic_id.set_index('cbg',inplace=True) ## 只有重新定义一列索引，后续与社区发现结果进行concat的时候，才能对得上
    X_deep.set_index('node',inplace=True) ## 只有重新定义一列索引，后续与社区发现结果进行concat的时候，才能对得上
    result=pd.concat([dic_id,X_deep],axis=1)


    result=result.fillna(0)
    result[result.isnull().T.any()]

    Xdeep_np=np.array(result)
    Xdeep = csr_matrix(Xdeep_np)
    print("Sparse matrix: \n",Xdeep)

    sparse.save_npz('D:/GNN/OCD/X_node2vec_'+INDEX+'.npz', Xdeep)  #保存


In [ ]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import preprocessing

import numpy as np

from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph

from gensim.models import Word2Vec

import warnings
import collections
from stellargraph import datasets
from IPython.display import display, HTML
import matplotlib.pyplot as plt

%matplotlib inline



In [37]:
OD_work=pd.read_csv('D:\\GNN\\OCD\\OD_work_pd.csv')
OD_weekend=pd.read_csv('D:\\GNN\\OCD\\OD_weekend_pd.csv')

In [38]:
ODtoA(OD_work,'work')
#ODtoX(OD,'work')


Sparse matrix: 
   (0, 1)	571
  (0, 2)	1466
  (0, 3)	55
  (0, 4)	17
  (0, 5)	40
  (0, 6)	8
  (0, 7)	10
  (0, 8)	25
  (0, 9)	7
  (0, 10)	2
  (0, 12)	285
  (0, 13)	92
  (0, 14)	69
  (0, 15)	164
  (0, 16)	50
  (0, 17)	39
  (0, 18)	151
  (0, 19)	51
  (0, 20)	203
  (0, 21)	6
  (0, 22)	23
  (0, 23)	1
  (0, 24)	1
  (0, 25)	2
  (0, 26)	1
  :	:
  (2084, 2052)	2
  (2084, 2055)	2
  (2084, 2056)	3
  (2084, 2057)	22
  (2084, 2058)	5
  (2084, 2063)	1
  (2084, 2065)	15
  (2084, 2066)	232
  (2084, 2067)	39
  (2084, 2068)	35
  (2084, 2069)	10
  (2084, 2070)	30
  (2084, 2071)	29
  (2084, 2072)	18
  (2084, 2073)	25
  (2084, 2074)	208
  (2084, 2075)	24
  (2084, 2076)	164
  (2084, 2077)	6
  (2084, 2078)	49
  (2084, 2079)	60
  (2084, 2080)	223
  (2084, 2081)	415
  (2084, 2082)	109
  (2084, 2083)	323


In [30]:
ODtoX_deepWalk(OD_work,'work')

100%|█████████████████████████████████████████████████████████████████████████████| 2085/2085 [00:05<00:00, 374.92it/s]


Finish


In [31]:
ODtoX_deepWalk(OD_weekend,'weekend')
ODtoA(OD_weekend,'weekend')

100%|█████████████████████████████████████████████████████████████████████████████| 2085/2085 [00:05<00:00, 385.22it/s]


Finish
Sparse matrix: 
   (0, 1)	313
  (0, 2)	759
  (0, 3)	33
  (0, 4)	10
  (0, 5)	24
  (0, 6)	7
  (0, 7)	8
  (0, 8)	14
  (0, 9)	9
  (0, 10)	3
  (0, 12)	151
  (0, 13)	71
  (0, 14)	50
  (0, 15)	95
  (0, 16)	30
  (0, 17)	19
  (0, 18)	92
  (0, 19)	48
  (0, 20)	99
  (0, 21)	1
  (0, 22)	5
  (0, 23)	1
  (0, 25)	1
  (0, 27)	3
  (0, 28)	3
  :	:
  (2084, 2055)	2
  (2084, 2056)	5
  (2084, 2057)	5
  (2084, 2058)	3
  (2084, 2059)	1
  (2084, 2061)	1
  (2084, 2065)	9
  (2084, 2066)	71
  (2084, 2067)	12
  (2084, 2068)	25
  (2084, 2069)	7
  (2084, 2070)	23
  (2084, 2071)	7
  (2084, 2072)	2
  (2084, 2073)	11
  (2084, 2074)	123
  (2084, 2075)	11
  (2084, 2076)	88
  (2084, 2077)	9
  (2084, 2078)	21
  (2084, 2079)	26
  (2084, 2080)	122
  (2084, 2081)	202
  (2084, 2082)	69
  (2084, 2083)	179


In [35]:
ODtoX_node2vec(OD_work,'work')

Number of random walks: 20850
Sparse matrix: 
   (0, 1)	0.0011778469197452068
  (0, 2)	0.0025947834365069866
  (0, 3)	-0.0009339596144855022
  (0, 4)	0.0016278126277029514
  (0, 5)	0.0003440091386437416
  (0, 6)	0.003684637136757374
  (0, 7)	0.002103361301124096
  (0, 8)	0.0026689539663493633
  (0, 9)	-0.002674838062375784
  (0, 10)	-0.0008863508701324463
  (0, 11)	-0.0011606835760176182
  (0, 12)	0.0038286936469376087
  (0, 13)	0.0020601479336619377
  (0, 14)	-0.0012016375549137592
  (0, 15)	-0.0033049150370061398
  (0, 16)	0.0028834184631705284
  (0, 17)	-0.003590826876461506
  (0, 18)	0.0005210433155298233
  (0, 19)	0.003217834047973156
  (0, 20)	-0.002950461581349373
  (0, 21)	0.003703739959746599
  (0, 22)	0.0036253919824957848
  (0, 23)	-0.0009435317479074001
  (0, 24)	-0.0015073041431605816
  (0, 25)	-0.003632251638919115
  :	:
  (2084, 232)	0.001791663933545351
  (2084, 233)	0.0018090996891260147
  (2084, 234)	-0.0026027769781649113
  (2084, 235)	0.002901854459196329
  (2084, 2